## ヒートマップでやりたいこと
- 縦bit, 横chain_strength, 値valid_y_num
- 縦bit, 横chain_strength, 値broken_chain_proportion

In [1]:
!pwd

In [2]:
import aws_QA_DwaveSampler_AutoEmbedding as aqa
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import statistics
import math
import datetime
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
print(datetime.datetime.now())

2021-01-14 20:05:03.825685


In [4]:
chain_strength_list = [5, 8, 10, 13, 15, 18, 20, 23, 25, 28, 30, 33, 35]
#chain_strength_dic = {20:10, 25:15, 30:20, 35:20, 40:30}

In [5]:
bit_list = [20, 25, 30, 35, 40]
bit_46_list = [20, 25, 30, 35, 40, 46]
num_list = [0, 1, 2, 3, 4]
n, num_reads = 1, 10000

In [6]:
broken_chain_proportion_dic = {}
valid_y_num_dic = {}
for bit in bit_46_list:
    broken_chain_proportion_dic[bit] = {}
    broken_chain_proportion_dic[bit] = {chain_strength:[] for chain_strength in chain_strength_list}
    valid_y_num_dic[bit] = {}
    valid_y_num_dic[bit] = {chain_strength:[] for chain_strength in chain_strength_list}

In [7]:
broken_chain_proportion_dic

{20: {5: [],
  8: [],
  10: [],
  13: [],
  15: [],
  18: [],
  20: [],
  23: [],
  25: [],
  28: [],
  30: [],
  33: [],
  35: []},
 25: {5: [],
  8: [],
  10: [],
  13: [],
  15: [],
  18: [],
  20: [],
  23: [],
  25: [],
  28: [],
  30: [],
  33: [],
  35: []},
 30: {5: [],
  8: [],
  10: [],
  13: [],
  15: [],
  18: [],
  20: [],
  23: [],
  25: [],
  28: [],
  30: [],
  33: [],
  35: []},
 35: {5: [],
  8: [],
  10: [],
  13: [],
  15: [],
  18: [],
  20: [],
  23: [],
  25: [],
  28: [],
  30: [],
  33: [],
  35: []},
 40: {5: [],
  8: [],
  10: [],
  13: [],
  15: [],
  18: [],
  20: [],
  23: [],
  25: [],
  28: [],
  30: [],
  33: [],
  35: []},
 46: {5: [],
  8: [],
  10: [],
  13: [],
  15: [],
  18: [],
  20: [],
  23: [],
  25: [],
  28: [],
  30: [],
  33: [],
  35: []}}

In [8]:
valid_y_num_dic

{20: {5: [],
  8: [],
  10: [],
  13: [],
  15: [],
  18: [],
  20: [],
  23: [],
  25: [],
  28: [],
  30: [],
  33: [],
  35: []},
 25: {5: [],
  8: [],
  10: [],
  13: [],
  15: [],
  18: [],
  20: [],
  23: [],
  25: [],
  28: [],
  30: [],
  33: [],
  35: []},
 30: {5: [],
  8: [],
  10: [],
  13: [],
  15: [],
  18: [],
  20: [],
  23: [],
  25: [],
  28: [],
  30: [],
  33: [],
  35: []},
 35: {5: [],
  8: [],
  10: [],
  13: [],
  15: [],
  18: [],
  20: [],
  23: [],
  25: [],
  28: [],
  30: [],
  33: [],
  35: []},
 40: {5: [],
  8: [],
  10: [],
  13: [],
  15: [],
  18: [],
  20: [],
  23: [],
  25: [],
  28: [],
  30: [],
  33: [],
  35: []},
 46: {5: [],
  8: [],
  10: [],
  13: [],
  15: [],
  18: [],
  20: [],
  23: [],
  25: [],
  28: [],
  30: [],
  33: [],
  35: []}}

## 本番：それぞれのbit_numにたいして2回、それぞれのbitデータに対して10回回し、データを得る

In [9]:
import aws_QA_DwaveSampler_AutoEmbedding as aqa
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import statistics
import math
import datetime
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
print(datetime.datetime.now())

2021-01-14 20:05:04.091225


In [11]:
chain_strength_list = [5, 8, 10, 13, 15, 18, 20, 23, 25, 28, 30, 33, 35]
bit_list = [20, 25, 30, 35, 40]
bit_46_list = [20, 25, 30, 35, 40, 46]
num_list = [0, 1, 2, 3, 4]
n, num_reads = 1, 10000


broken_chain_proportion_dic = {}
valid_y_num_dic = {}
for bit in bit_46_list:
    broken_chain_proportion_dic[bit] = {}
    broken_chain_proportion_dic[bit] = {chain_strength:[] for chain_strength in chain_strength_list}
    valid_y_num_dic[bit] = {}
    valid_y_num_dic[bit] = {chain_strength:[] for chain_strength in chain_strength_list}


for jj in range(2):
    for bit in bit_46_list:
        if bit != 46:
            for num in num_list:
                path = '../input/ost{}_{}.csv'.format(bit, num)
                df = pd.read_csv(path, sep=',')
                bqm = aqa.make_Hamiltonian(df)

                for chain_strength in chain_strength_list:
                    res = aqa.aws_get_response(bqm, chain_strength, num_reads)
                    ads = aqa.aws_DSampler(df, bqm, num_reads, chain_strength, res)
                    valid_y_info_dic = ads.find_valid_y()

                    if len(valid_y_info_dic) > 0:
                        broken_list = [valid_y_info[1] for valid_y_info in list(valid_y_info_dic.values())]
                        broken_chain_proportion_dic[bit][chain_strength].append(statistics.mean(broken_list))
                        valid_y_num_dic[bit][chain_strength].append(len(valid_y_info_dic))
        else:
            path = '../input/ost{}.csv'.format(bit)
            df = pd.read_csv(path, sep=',')
            bqm = aqa.make_Hamiltonian(df)

            for chain_strength in chain_strength_list:
                res = aqa.aws_get_response(bqm, chain_strength, num_reads)
                ads = aqa.aws_DSampler(df, bqm, num_reads, chain_strength, res)
                valid_y_info_dic = ads.find_valid_y()

                if len(valid_y_info_dic) > 0:
                    broken_list = [valid_y_info[1] for valid_y_info in list(valid_y_info_dic.values())]
                    broken_chain_proportion_dic[bit][chain_strength].append(statistics.mean(broken_list))
                    valid_y_num_dic[bit][chain_strength].append(len(valid_y_info_dic))
        
        print('{} bit finished'.format(bit))

20 bit finished
25 bit finished
30 bit finished
35 bit finished
40 bit finished
46 bit finished
20 bit finished
25 bit finished
30 bit finished
35 bit finished
40 bit finished
46 bit finished


In [12]:
broken_chain_proportion_dic

{20: {5: [0.3, 0.3, 0.25, 0.05, 0.43333333333333335, 0.15],
  8: [0.07236842105263158,
   0.0033333333333333335,
   0.013500000000000002,
   0.002777777777777778,
   0.003448275862068966,
   0.02457983193277311,
   0.0024324324324324327,
   0.013571428571428573,
   0.01058091286307054,
   0.028],
  10: [0.009661016949152543,
   0.00037313432835820896,
   0.004121863799283154,
   0.002857142857142857,
   0.0030303030303030303,
   0.005808080808080808,
   0.0007575757575757576,
   0.0018032786885245902,
   0.0030927835051546395,
   0.003125],
  13: [0.0017543859649122807,
   0.000980392156862745,
   0.0017857142857142859,
   0.0,
   0.001851851851851852,
   0.0028704939919893192,
   0.0004950495049504951,
   0.0004120879120879121,
   0.0004761904761904762,
   0.0025],
  15: [0.0006723716381418093,
   0.0008695652173913044,
   0.0016990291262135924,
   0.00045045045045045046,
   0.0,
   0.00101010101010101,
   0.0,
   0.0008992805755395684,
   0.0018181818181818182,
   0.0],
  18: [0.0004

In [13]:
valid_y_num_dic

{20: {5: [5, 1, 1, 1, 3, 1],
  8: [38, 60, 100, 144, 29, 238, 185, 70, 241, 25],
  10: [295, 134, 279, 210, 33, 198, 66, 305, 194, 32],
  13: [627, 51, 224, 168, 27, 749, 101, 364, 105, 20],
  15: [818, 115, 206, 222, 16, 891, 45, 278, 165, 21],
  18: [526, 54, 217, 126, 44, 778, 42, 279, 48, 16],
  20: [407, 80, 336, 187, 47, 413, 39, 158, 98, 20],
  23: [454, 78, 198, 56, 36, 606, 63, 215, 91, 6],
  25: [458, 37, 122, 67, 47, 748, 104, 140, 172, 18],
  28: [524, 40, 170, 63, 10, 332, 27, 104, 98, 7],
  30: [571, 56, 139, 78, 12, 322, 36, 131, 106, 10],
  33: [282, 31, 158, 194, 9, 351, 113, 64, 108, 29],
  35: [349, 25, 132, 84, 14, 376, 29, 79, 37, 12]},
 25: {5: [9],
  8: [5, 3, 3, 6, 8, 28],
  10: [25, 153, 5, 1, 199, 4, 421, 32, 2, 8],
  13: [74, 304, 92, 8, 131, 33, 245, 90, 6, 124],
  15: [39, 132, 53, 7, 114, 42, 224, 19, 8, 245],
  18: [37, 244, 22, 1, 56, 18, 202, 16, 14, 48],
  20: [15, 56, 17, 8, 129, 210, 364, 23, 6, 91],
  23: [39, 130, 18, 11, 31, 98, 105, 10, 2, 93],
 

In [14]:
broken_chain_proportion_46 = broken_chain_proportion_dic[46]
valid_y_num_46 = valid_y_num_dic[46]

In [15]:
broken_chain_proportion_46

{5: [],
 8: [],
 10: [],
 13: [],
 15: [],
 18: [0.21739130434782608],
 20: [0.10869565217391304],
 23: [0.03260869565217391],
 25: [0.008695652173913044, 0.028985507246376812],
 28: [0.0],
 30: [0.0],
 33: [0.0, 0.043478260869565216],
 35: [0.021739130434782608, 0.021739130434782608]}

In [16]:
valid_y_num_46

{5: [],
 8: [],
 10: [],
 13: [],
 15: [],
 18: [1],
 20: [1],
 23: [2],
 25: [5, 3],
 28: [1],
 30: [1],
 33: [1, 1],
 35: [1, 1]}

In [17]:
del broken_chain_proportion_dic[46]

In [18]:
broken_chain_proportion_dic.keys()

dict_keys([20, 25, 30, 35, 40])

In [19]:
del valid_y_num_dic[46]

In [20]:
valid_y_num_dic.keys()

dict_keys([20, 25, 30, 35, 40])

In [21]:
## 5が出ないのが多いので、削除
for bit in bit_list:
    del broken_chain_proportion_dic[bit][5]
    print(broken_chain_proportion_dic[bit].keys()==[10, 15, 20, 25, 30, 35])#間違えた。得られた辞書自体に問題はない

False
False
False
False
False


In [22]:
broken_chain_proportion_dic

{20: {8: [0.07236842105263158,
   0.0033333333333333335,
   0.013500000000000002,
   0.002777777777777778,
   0.003448275862068966,
   0.02457983193277311,
   0.0024324324324324327,
   0.013571428571428573,
   0.01058091286307054,
   0.028],
  10: [0.009661016949152543,
   0.00037313432835820896,
   0.004121863799283154,
   0.002857142857142857,
   0.0030303030303030303,
   0.005808080808080808,
   0.0007575757575757576,
   0.0018032786885245902,
   0.0030927835051546395,
   0.003125],
  13: [0.0017543859649122807,
   0.000980392156862745,
   0.0017857142857142859,
   0.0,
   0.001851851851851852,
   0.0028704939919893192,
   0.0004950495049504951,
   0.0004120879120879121,
   0.0004761904761904762,
   0.0025],
  15: [0.0006723716381418093,
   0.0008695652173913044,
   0.0016990291262135924,
   0.00045045045045045046,
   0.0,
   0.00101010101010101,
   0.0,
   0.0008992805755395684,
   0.0018181818181818182,
   0.0],
  18: [0.00047528517110266165,
   0.0,
   0.0002304147465437788,
   0

In [23]:
## 5が出ないのが多いので、削除
for bit in bit_list:
    del valid_y_num_dic[bit][5]
    print(valid_y_num_dic[bit].keys()==[10, 15, 20, 25, 30, 35])#間違えた。得られた辞書自体に問題はない

False
False
False
False
False


In [24]:
valid_y_num_dic#15, 15, 15, 20, 20

{20: {8: [38, 60, 100, 144, 29, 238, 185, 70, 241, 25],
  10: [295, 134, 279, 210, 33, 198, 66, 305, 194, 32],
  13: [627, 51, 224, 168, 27, 749, 101, 364, 105, 20],
  15: [818, 115, 206, 222, 16, 891, 45, 278, 165, 21],
  18: [526, 54, 217, 126, 44, 778, 42, 279, 48, 16],
  20: [407, 80, 336, 187, 47, 413, 39, 158, 98, 20],
  23: [454, 78, 198, 56, 36, 606, 63, 215, 91, 6],
  25: [458, 37, 122, 67, 47, 748, 104, 140, 172, 18],
  28: [524, 40, 170, 63, 10, 332, 27, 104, 98, 7],
  30: [571, 56, 139, 78, 12, 322, 36, 131, 106, 10],
  33: [282, 31, 158, 194, 9, 351, 113, 64, 108, 29],
  35: [349, 25, 132, 84, 14, 376, 29, 79, 37, 12]},
 25: {8: [5, 3, 3, 6, 8, 28],
  10: [25, 153, 5, 1, 199, 4, 421, 32, 2, 8],
  13: [74, 304, 92, 8, 131, 33, 245, 90, 6, 124],
  15: [39, 132, 53, 7, 114, 42, 224, 19, 8, 245],
  18: [37, 244, 22, 1, 56, 18, 202, 16, 14, 48],
  20: [15, 56, 17, 8, 129, 210, 364, 23, 6, 91],
  23: [39, 130, 18, 11, 31, 98, 105, 10, 2, 93],
  25: [14, 148, 18, 60, 57, 126, 36,

In [25]:
new_chain_strength_list = [10, 15, 20, 25, 30, 35]
#bit_list = [20, 25, 30, 35, 40]

In [26]:
mean_broken_chain_proportion_dic = {}
for bit in bit_list:
    mean_broken_chain_proportion_dic[bit] = {}
    for chain_strength in new_chain_strength_list:
        mean_broken_chain_proportion_dic[bit][chain_strength] = statistics.mean(broken_chain_proportion_dic[bit][chain_strength])

StatisticsError: mean requires at least one data point

In [ ]:
mean_broken_chain_proportion_dic

In [ ]:
mean_broken_chain_proportion_list = [list(mean_broken_chain_proportion_dic[bit].values()) for bit in bit_list]

In [ ]:
mean_broken_chain_proportion_list

In [ ]:
mean_valid_y_num_dic = {}
for bit in bit_list:
    mean_valid_y_num_dic[bit] = {}
    for chain_strength in new_chain_strength_list:
        mean_valid_y_num_dic[bit][chain_strength] = statistics.mean(valid_y_num_dic[bit][chain_strength])

In [ ]:
mean_valid_y_num_dic#20bit-15, 25bit-15, 30bit-15, 35bit-20, 40bit-10, 

In [ ]:
mean_valid_y_num_list = [list(mean_valid_y_num_dic[bit].values()) for bit in bit_list]

In [ ]:
mean_valid_y_num_list

In [ ]:
import seaborn as sns

names = ['{}bit'.format(i) for i in bit_list]

sns.heatmap(mean_broken_chain_proportion_list, xticklabels=new_chain_strength_list, yticklabels=names)

plt.xlabel('chain strength')
plt.ylabel('dataset name')
plt.title('broken chain proportion')

plt.tight_layout()
plt.savefig('__broken_chain_proportion.png')
plt.show()

In [ ]:
import seaborn as sns

names = ['{}bit'.format(i) for i in bit_list]

sns.heatmap(mean_valid_y_num_list, xticklabels=new_chain_strength_list, yticklabels=names)

plt.xlabel('chain strength')
plt.ylabel('dataset name')
plt.title('number of valid sample')

plt.tight_layout()
plt.savefig('__chain_strength_valid_y_num')
plt.show()

## chain_strength_dic = {20:15, 25:15, 30:15, 35:20, 40:20}と決定

In [ ]:
bit = 46
for jj in range(3):
    path = '../input/ost{}.csv'.format(bit)
    df = pd.read_csv(path, sep=',')
    bqm = aqa.make_Hamiltonian(df)

    for chain_strength in chain_strength_list:
        res = aqa.aws_get_response(bqm, chain_strength, num_reads)
        ads = aqa.aws_DSampler(df, bqm, num_reads, chain_strength, res)
        valid_y_info_dic = ads.find_valid_y()

        if len(valid_y_info_dic) > 0:
            broken_list = [valid_y_info[1] for valid_y_info in list(valid_y_info_dic.values())]
            broken_chain_proportion_46[chain_strength].append(statistics.mean(broken_list))
            valid_y_num_46[chain_strength].append(len(valid_y_info_dic))
        
    print('{} time finished'.format(bit))#また位置間違えた

In [ ]:
broken_chain_proportion_46

In [ ]:
valid_y_num_46

In [ ]:
#20bit
